In [7]:
%pip install youtube-transcript-api


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

# replace 'video_id' with your youtube video id
video_id = '05-RrWGBcdY&ab'
transcript = YouTubeTranscriptApi.get_transcript(video_id)

print(transcript)

[{'text': 'is it fair to say that you are at the', 'start': 0.439, 'duration': 6.541}, {'text': 'moment in the form of your life', 'start': 3.54, 'duration': 3.44}, {'text': "I think it's fair to say that I'm in the", 'start': 7.34, 'duration': 4.62}, {'text': 'foreign', 'start': 9.72, 'duration': 2.24}, {'text': 'how do you say it not stable but quite', 'start': 39.42, 'duration': 6.479}, {'text': "don't don't have weeks where you cannot", 'start': 43.399, 'duration': 5.98}, {'text': 'train because of injuries or food I had', 'start': 45.899, 'duration': 4.621}, {'text': 'a really good winter and I had no', 'start': 49.379, 'duration': 3.5}, {'text': 'injuries', 'start': 50.52, 'duration': 2.359}, {'text': 'in a good shape and then uh you start', 'start': 57.739, 'duration': 7.32}, {'text': 'building towards the tour yeah', 'start': 61.5, 'duration': 3.559}, {'text': 'think of its capitulation yesterday', 'start': 69.86, 'duration': 4.079}, {'text': "uh he's yeah complete he's such a"

In [9]:
from pytube import YouTube

def get_thumbnail(url):
    yt = YouTube(url)
    thumbnail_url = yt.thumbnail_url
    return thumbnail_url

In [10]:
thumbnail_url = get_thumbnail("https://www.youtube.com/watch?v=ferZnZ0_rSM&ab_channel=NPRMusic")
thumbnail_url

'https://i.ytimg.com/vi/ferZnZ0_rSM/hq720.jpg?sqp=-oaymwEXCNUGEOADIAQqCwjVARCqCBh4INgESFo&rs=AOn4CLCi03kD2pWK6ZZMSrLAK3WikzErVw'

In [47]:
yt = YouTube("https://www.youtube.com/watch?v=RmVmqvqUWzI&ab_channel=TLDRNewsEU")
yt.title, yt.length, yt.channel_id

('Ukraine Advances Around Bakhmut: What Next?',
 551,
 'UC-eegKVWEgBCa4OzjnK_PtA')

In [44]:
help(yt)

Help on YouTube in module pytube.__main__ object:

class YouTube(builtins.object)
 |  YouTube(url: str, on_progress_callback: Optional[Callable[[Any, bytes, int], NoneType]] = None, on_complete_callback: Optional[Callable[[Any, Optional[str]], NoneType]] = None, proxies: Dict[str, str] = None, use_oauth: bool = False, allow_oauth_cache: bool = True)
 |  
 |  Core developer interface for pytube.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, o: object) -> bool
 |      Return self==value.
 |  
 |  __init__(self, url: str, on_progress_callback: Optional[Callable[[Any, bytes, int], NoneType]] = None, on_complete_callback: Optional[Callable[[Any, Optional[str]], NoneType]] = None, proxies: Dict[str, str] = None, use_oauth: bool = False, allow_oauth_cache: bool = True)
 |      Construct a :class:`YouTube <YouTube>`.
 |      
 |      :param str url:
 |          A valid YouTube watch URL.
 |      :param func on_progress_callback:
 |          (Optional) User defined callback function for 

In [12]:
url = "https://www.youtube.com/watch?v=RmVmqvqUWzI&ab_channel=TLDRNewsEU"
id = url.split("watch?v=")[1].split("&")[0]
id.split("&")[0]

'RmVmqvqUWzI'

In [13]:
url = "https://www.youtube.com/watch?v=RmVmqvqUWzI&ab_channel=TLDRNewsEU"
id = url.split("watch?v=")[1].split("&")[0]

yt = YouTube("https://www.youtube.com/watch?v=RmVmqvqUWzI&ab_channel=TLDRNewsEU")

transcript = YouTubeTranscriptApi.get_transcript(id)

text = f"TITLE: {yt.title} TRANSCRIPT: {transcript}"

In [30]:
import os
import sys
import openai

from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.schema import AIMessage, HumanMessage

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

sys.path.insert(0, ".")
sys.path.insert(1, "..")
from src.config import AZURE_OPENAI_KEY, AZURE_OPENAI_ENDPOINT, AZURE_ENGINE_NAME

In [15]:
openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

In [17]:
model = ChatOpenAI(engine=AZURE_ENGINE_NAME, openai_api_key=AZURE_OPENAI_KEY, openai_api_base=AZURE_OPENAI_ENDPOINT, temperature=0.5)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [18]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["transcription"],
            template="I am giving you a youtube transciption and the title {transcription} could you summary the video content with less than 100 words?",
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
first_chain = LLMChain(llm=model, prompt=chat_prompt_template)
structured_text = first_chain.run(text)

In [19]:
structured_text

'The video discusses the current situation in Ukraine, focusing on the advances made by the Ukrainian Army around Bakhmut. The front lines have been relatively stable, but Ukraine has been making small gains in strategic areas. They have shifted to a more attritional strategy and received support from American custom munitions. The Russians have tried to counter these advances but have faced limited success. The capture of Bakhmut would be symbolically significant for Ukraine. The video also mentions the availability of other conflict-related content on Nebula, a streaming service.'

In [42]:
start_message = "I'm providing you with a YouTube transcription and the title: {transcription}."
prompt_start = """Please generate an output in JSON format. If a requested field doesn't make sense, please fill it with 'NA'. The maximum word count for each field is indicated after each field placeholder: 
                {{
                    "Topic": "A topic derived from the video transcription, intended for use in a recommendation system (up to 10 words)",
                    "Summary": "A summary of the video transcription (up to 100 words)",
                    "Wikipedia": "Keyword to be used for deeper research on Wikipedia relating to the video (up to 5 words)",
                    "Google": "Keyword to be used for further searches on Google about the video (up to 5 words)",
                    "Arxiv": "Keyword to search for academic papers related to the video's topic (up to 5 words)"
                }}

                Transcription:
                {text}
            """
final_prompt = prompt_start.format(text=text)

prompt = [AIMessage(content=start_message), HumanMessage(content=final_prompt)]
model
response = model(prompt).content

In [43]:
response

'{\n    "Topic": "Ukraine Advances Around Bakhmut: What Next?",\n    "Summary": "Not much is happening along the front lines in Ukraine at the moment. The Ukrainian Army has moved away from large-scale offensives towards a more attritional strategy. However, they have continued to make small but strategically significant gains in Bakhmut, prompting anxiety among Russian channels and recently mobilized Russian troops. This video provides updates on the battlefield and explains the reasons for the slower pace of the Ukrainian advance. It also highlights the progress made in the Northeast around Kupiansk and Kramatorsk. The video emphasizes the symbolic value of Bakhmut and its potential impact on the conflict.",\n    "Wikipedia": "Ukraine, Bakhmut",\n    "Google": "Ukraine advances, Bakhmut",\n    "Arxiv": "Ukraine conflict, Bakhmut"\n}'